In [2]:
import os
import pandas as pd
import csv

##initialize first dataframe
df = pd.read_excel('combinepdf_clean.xlsx')



# Remove unused metric of tournament totals
mask = ~df['R'].astype(str).str.startswith('Tournament Totals:')
df = df.loc[mask]

##if team names are needed
p = True
if p == True:
    output_path = "./data_team.csv"
    df.to_csv(output_path, index=False)
    
    
    
    
    
# Remove Team names as not needed
mask = ~df['R'].astype(str).str.startswith('Team:')
df = df.loc[mask]

##Drop random column
df = df.drop(columns =['Unnamed: 8'])



#Remove repeat columns from aggregation
# mask = ~df['R'].astype(str).str.startswith('R')
# df = df.loc[mask]
## drop rows with null values
df = df.dropna(how='any')
# df = df.iloc[1:]

    
output_path = "./data.csv"
df.to_csv(output_path, index=False)
##IMPORTANT LOOK AT THESE potential MANUAL INSTRUCTIONS BELOW
##MANUALLY REMOVE HEADER ROW HERE BEFORE PRE-Processing
##Delete certain ranges in the resulting file, weird data that is inconsistent with everything else



In [3]:
import csv

with open('data.csv', 'r', encoding='utf-8') as csvfile:
    csvreader = csv.reader(csvfile)
    ##skip useless header row
    row = next(csvreader)
    
    ##Dictionary defines names as keys and a value of a 'list' with specified measurements
    ##[total rounds, total speaks, total ranks, average speaks, average ranks, overall win/loss ratio, gov win/loss ratio, opp. win/loss ratio]
    ##during parsing, overall win/loss and the indices onward will hold its own list to hold the needed intermediate values for final calc. at end
    db = dict()
    row = next(csvreader)
    p = 1
    while True:
        try:
            while (p>=5076 and p<5176) or (p>=5475 and p<5566):
                row = next(csvreader)
            if row[0] == 'R':
                name1 = row[5]
                ##remove newline characters
                name1 = name1.replace('\n', ' ')
                if name1 not in db:
                    db.update({name1: [0, 0, 0, 0, 0, [0 , 0], [0, 0], [0, 0]]})
                    db[name1][0] = 0
                # if name1 == 'Ann Tran (N)':
                #     print("hi")
                
                name2 = row[6]
                name2 = name2.replace('\n', ' ')
                if name2 not in db:
                    db.update({name2: [0, 0, 0, 0, 0, [0 , 0], [0, 0], [0, 0]]})
                    db[name2][0] = 0                    
                
                ##iterate actual table information
                row = next(csvreader)
                p+=1
                while row[0] != 'R':
                    ##update total rounds
                    db[name1][0] +=1
                    db[name2][0] +=1

                    ##update total speaks and total ranks
                    speak_rank1 = row[5][1:-1]
                    speak_rank2 = row[6][1:-1]
                    
                    ##parse the strings further,
                    sprlist1 = speak_rank1.split(",")
                    sprlist2 = speak_rank2.split(",")
                    ##convert them to integers
                    sprlist1 = [float(x) for x in sprlist1]
                    sprlist2 = [float(x) for x in sprlist2]


                    ##update total speaks and total ranks with the extracted data
                    ##speaks
                    db[name1][1]+=sprlist1[0]
                    db[name2][1]+=sprlist2[0]

                    ##ranks
                    db[name1][2]+=sprlist1[1]
                    db[name2][2]+=sprlist2[1]

                    ##update intermediate win and loss
                    if row[2] == 'W':
                        db[name1][5][0]+=1
                        db[name2][5][0]+=1
                        if row[1] == 'G':
                            ##update gov win
                            db[name1][6][0]+=1
                            db[name2][6][0]+=1
                        else:
                            ##update opp win
                            db[name1][7][0]+=1
                            db[name2][7][0]+=1
                    else:
                        db[name1][5][1]+=1
                        db[name2][5][1]+=1
                        if row[1] == 'G':
                            ##update gov loss
                            db[name1][6][1]+=1
                            db[name2][6][1]+=1
                        else:
                            ##update opp loss
                            db[name1][7][1]+=1
                            db[name2][7][1]+=1
                    ##next row
                    row = next(csvreader)
                    p+=1
                
        except StopIteration:
            # Reached the end of the file
            break
    
    ##print(db)
        
        
    ##update the average metrics with total rounds and other fields, and win/loss ratios as well
    for entry in db:
        ##average speaks
        try:
            db[entry][3] = round(db[entry][1]/db[entry][0], 2)
        except ZeroDivisionError:
            db[entry][3] = round(db[entry][1], 2)
        
        ##average ranks
        try:
            db[entry][4] = round(db[entry][2]/db[entry][0], 2)
        except:
            db[entry][4] = round(db[entry][2], 2)
        
        ##overall win/loss ratio
        try:
            db[entry][5] = round(db[entry][5][0]/db[entry][5][1], 2)
        except:
            db[entry][5] = round(db[entry][5][0], 2)
        ##gov win/loss ratio
        try:
            db[entry][6] = round(db[entry][6][0]/db[entry][6][1], 2)
        except ZeroDivisionError:
            db[entry][6] = round(db[entry][6][0], 2)
        
        ##opp win/loss ratio
        try: 
            db[entry][7] = round(db[entry][7][0]/db[entry][7][1], 2)
        except ZeroDivisionError:
            db[entry][7] = round(db[entry][7][0], 2)
      
                                
    print(db)

{'Bart Kaminski (V)': [15, 455.0, 29.0, 30.33, 1.93, 1.5, 1.67, 1.33], 'Eugenie Park (V)': [44, 1318.0, 88.0, 29.95, 2.0, 1.44, 2.83, 0.75], 'Andrew Monteith (V)': [25, 750.0, 49.0, 30.0, 1.96, 1.5, 1.5, 1.5], 'Peregrine Beckett (V)': [5, 141.0, 13.0, 28.2, 2.6, 1.5, 0.5, 2], 'Irina Znamirowski (V)': [30, 872.0, 71.0, 29.07, 2.37, 1.14, 0.88, 1.5], 'John Lowe (V)': [5, 143.0, 14.0, 28.6, 2.8, 0.67, 0.0, 2], 'Ahmad Howard (V)': [39, 1175.0, 70.0, 30.13, 1.79, 3.88, 4.5, 3.25], 'Helen W (V)': [5, 145.0, 12.0, 29.0, 2.4, 1.5, 1.0, 2.0], 'Aidan Shannon (V)': [40, 1133.0, 93.0, 28.32, 2.33, 0.9, 1.0, 0.83], 'Nish Sinha (V)': [30, 845.0, 84.0, 28.17, 2.8, 1.0, 0.78, 1.33], 'Katrina Hotaling (V)': [50, 1556.0, 88.0, 31.12, 1.76, 4.0, 3.67, 4.5], 'Nick DeVito (V)': [34, 1070.0, 55.0, 31.47, 1.62, 7.5, 5.33, 14.0], 'Gabe Ritter (V)': [15, 461.0, 31.0, 30.73, 2.07, 4.0, 3.5, 5.0], 'Prottoya Chowdhury (V)': [10, 276.0, 32.0, 27.6, 3.2, 1.5, 4.0, 0.67], 'Leo Fleissig (V)': [29, 836.0, 66.0, 28.83,

In [4]:
##for storing teams names and what their opponents 
with open('data_team.csv', 'r', encoding='utf-8') as csvfile:
    csvreader = csv.reader(csvfile)
    ##skip useless header row
    row = next(csvreader)
    row = next(csvreader)
    ##Key is team name and value is a list where index 0 is member1 and index 1 is member 2, index 2 is a dictionary of the following form:
    ##NOT USED##{'G': [(num wins), (num losses)], 'O': [(num wins), (num losses)]}
    db_teams = dict()
    while True:
        try: 
            if row[0][0:5] == 'Team:' and row[0][6:] != "":
                team_name = row[0][6:]
                row = next(csvreader)
                if row[0] == 'R':
                    name1 = row[5]
                    ##remove newline characters
                    name1 = name1.replace('\n', ' ')

                    name2 = row[6]
                    name2 = name2.replace('\n', ' ')
                    ##update team name
                    if team_name not in db_teams:
                        db_teams.update({team_name: [name1, name2]})
                        # db_teams[2].update({'G': [0, 0]})
                        # db_teams[2].update({'O': [0, 0]})
            row = next(csvreader)
                   
                
        except StopIteration:
            break
    
    print(db_teams)

{'(B)ART HISTORY': ['Bart Kaminski (V)', 'Eugenie Park (V)'], 'pink crocks no socks III': ['Andrew Monteith (V)', 'Peregrine Beckett (V)'], '(max character limit exceeded)': ['Irina Znamirowski (V)', 'John Lowe (V)'], 'Sam and Laura': ['Ahmad Howard (V)', 'Helen W (V)'], 'Chicago SS': ['Aidan Shannon (V)', 'Nish Sinha (V)'], "where are APDA's femboys?": ['Katrina Hotaling (V)', 'Nick DeVito (V)'], 'Brown CR': ['Gabe Ritter (V)', 'Prottoya Chowdhury (V)'], 'Brown Hyperreality': ['Leo Fleissig (V)', 'Mac Hays (V)'], 'Brandeis CS': ['Ethan Shurberg (V)', 'Shiv Chawla (V)'], 'Ahmad and Helen': ['Laura Giugno (V)', 'Samuel Johnson (V)'], 'Stanford CZ': ['Amy Chang (N)', 'Vivian Zhu (V)'], 'Stanford AK': ['Anshika Agarwal (N)', 'Joseph Kim (N)'], 'Carry me, Vinny T!': ['Abraham Paik (V)', 'Vincent Tocci (N)'], 'Williams D&D': ['Dilay Kolinoglu (N)', 'Dilay Namli (N)'], 'Williams GM': ['Gautam Ramasamy (N)', 'Margaret MacGillivray (N)'], 'CornHub': ['Connor Beaney (N)', 'Irina Znamirowski (V)

In [17]:
##last sweep thorugh data
##prediction set of features #1
with open('data.csv', 'r', encoding='utf-8') as csvfile, open('output.csv', 'w', newline='') as output_file:
    csvreader = csv.reader(csvfile)
    writer = csv.writer(output_file)
    
    # Write the header row of output file
    ##P1 and P2 vs P3 and P4
    writer.writerow(['P1 Avg. Speak', 'P2 Avg. Speak', 'P1 Avg. Rank', 'P2 Avg. Rank', 'P1 W:L', 'P2 W:L', \
                     'P3 Avg. Speak', 'P4 Avg. Speak', 'P3 Avg. Rank', 'P4 Avg. Rank', \
                    'P3 W:L', 'P4 W:L', 'Winner G/O'])

    ##skip useless header row
    row = next(csvreader)
    
    ##first actual row
    row = next(csvreader)
    while True:
        try: 
            if row[0] == 'R':
                name1 = row[5]
                ##remove newline characters
                name1 = name1.replace('\n', ' ')

                name2 = row[6]
                name2 = name2.replace('\n', ' ')
                
                
                row = next(csvreader)
                while row[0] != 'R':
                    if row[2] == 'W':
                        win_side = row[1]
                        team_opp = row[3]
                        ##has list of names
                        names = db_teams.get(team_opp)
                        
                        if names == None:
                            row = next(csvreader)
                            continue
                        
                        lose_side = ""
                        if win_side == 'G':
                            lose_side = 'O'
                        else:
                            lose_side = 'G'
                        
                        other_name1 = names[0]
                        other_name2 = names[1]
                        
                        stat_name1 = db.get(name1)
                        stat_name2 = db.get(name2)
                        stat_other_name1 = db.get(other_name1)
                        stat_other_name2 = db.get(other_name2)
                        if stat_name1 == None or stat_name2 == None or stat_other_name1 == None or stat_other_name2 == None:
                            row = next(csvreader)
                            continue
                        
                        
                        writer.writerow([stat_name1[3], stat_name2[3], stat_name1[4], stat_name2[4], stat_name1[5], stat_name2[5], win_side, \
                                       stat_other_name1[3], stat_other_name2[3], stat_other_name1[4], stat_other_name2[4], \
                                         stat_other_name1[5], stat_other_name2[5], lose_side, win_side])
                        
                        
                        
                    
                    
                    
                    
                    row = next(csvreader)


        except StopIteration:
            break


In [19]:
##Prediction set of features #2
with open('data.csv', 'r', encoding='utf-8') as csvfile, open('output.csv', 'w', newline='') as output_file:
    csvreader = csv.reader(csvfile)
    writer = csv.writer(output_file)
    
    # Write the header row of output file
    ##P1 and P2 vs P3 and P4
    writer.writerow(['P1 Avg. Speak', 'P2 Avg. Speak', 'P1 Avg. Rank', 'P2 Avg. Rank', 'P1 W:L', 'P2 W:L', 'Side 1', \
                     'P3 Avg. Speak', 'P4 Avg. Speak', 'P3 Avg. Rank', 'P4 Avg. Rank', \
                    'P3 W:L', 'P4 W:L', 'Side 2', 'Winner'])
                     ##Winner P1-P2/P3-P4 columned defined as 0 for P1-P2 and 1 for P3-P4

    ##skip useless header row
    row = next(csvreader)
    
    ##first actual row
    row = next(csvreader)
    while True:
        try: 
            if row[0] == 'R':
                name1 = row[5]
                ##remove newline characters
                name1 = name1.replace('\n', ' ')

                name2 = row[6]
                name2 = name2.replace('\n', ' ')
                
                
                row = next(csvreader)
                while row[0] != 'R':
                    result = row[2]
                    # if result == 'W':
                    #      result = 0
                    # else:
                    #     result = 1


                    team1_side = row[1]
                    team2_side = ""
                    if team1_side == 'G':
                        team2_side = 'O'
                    else:
                        team2_side = 'G'
                    

                    team_opp = row[3]
                    ##has list of names
                    names = db_teams.get(team_opp)

                    if names == None:
                        row = next(csvreader)
                        continue


                    other_name1 = names[0]
                    other_name2 = names[1]

                    stat_name1 = db.get(name1)
                    stat_name2 = db.get(name2)
                    stat_other_name1 = db.get(other_name1)
                    stat_other_name2 = db.get(other_name2)
                    if stat_name1 == None or stat_name2 == None or stat_other_name1 == None or stat_other_name2 == None:
                        row = next(csvreader)
                        continue


                    writer.writerow([stat_name1[3], stat_name2[3], stat_name1[4], stat_name2[4], stat_name1[5], stat_name2[5], team1_side, \
                                   stat_other_name1[3], stat_other_name2[3], stat_other_name1[4], stat_other_name2[4], \
                                     stat_other_name1[5], stat_other_name2[5], team2_side, result])







                    row = next(csvreader)


        except StopIteration:
            break


In [15]:
##ALGORITHM DECISION TREE IMPLEMENTATION
##Prediction set of features #1
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics


##Read in csv
df = pd.read_csv('output.csv')

##target variable
var = df['Winner G/O']

names = ['P1 Avg. Speak', 'P2 Avg. Speak', 'P1 Avg. Rank', 'P2 Avg. Rank', 'P1 W:L', 'P2 W:L', \
         'P3 Avg. Speak', 'P4 Avg. Speak', 'P3 Avg. Rank', 'P4 Avg. Rank', 'P3 W:L', 'P4 W:L']

##print(df[names])

X = pd.get_dummies(df[names])


# Split into training and test set
X_train, X_test, y_train, y_test = train_test_split(
            X, var, test_size = 0.25, random_state=42)

tree = DecisionTreeClassifier()
tree.fit(X_train, y_train)

##Predict and evaluate
y_pred = tree.predict(X_test)


print("Accuracy of Tree:", metrics.accuracy_score(y_test, y_pred))
print(metrics.confusion_matrix(y_test, y_pred))
print("")


tree_rules = export_text(tree, feature_names=list(X_train))
print("")
print(tree_rules)




Accuracy of Tree: 1.0
[[246   0]
 [  0 214]]


|--- P1-P2 Side_G <= 0.50
|   |--- class: O
|--- P1-P2 Side_G >  0.50
|   |--- class: G



In [20]:
##ALGORITHM DECISION TREE IMPLEMENTATION
##Prediction set of features #2
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics


##Read in csv
df = pd.read_csv('output.csv')

##target variable
var = df['Winner P1-P2/P3-P4']

names = ['P1 Avg. Speak', 'P2 Avg. Speak', 'P1 Avg. Rank', 'P2 Avg. Rank', 'P1 W:L', 'P2 W:L', 'Side 1', \
         'P3 Avg. Speak', 'P4 Avg. Speak', 'P3 Avg. Rank', 'P4 Avg. Rank', 'P3 W:L', 'P4 W:L', 'Side 2']

##print(df[names])

X = pd.get_dummies(df[names])


# Split into training and test set
X_train, X_test, y_train, y_test = train_test_split(
            X, var, test_size = 0.25, random_state=42)

tree = DecisionTreeClassifier()
tree.fit(X_train, y_train)

##Predict and evaluate
y_pred = tree.predict(X_test)


print("Accuracy of Tree:", metrics.accuracy_score(y_test, y_pred))
print(metrics.confusion_matrix(y_test, y_pred))
print("")


tree_rules = export_text(tree, feature_names=list(X_train))
print("")
print(tree_rules)



Accuracy of Tree: 0.7220447284345048
[[335 103]
 [158 343]]


|--- P1 W:L <= 0.89
|   |--- P3 W:L <= 0.47
|   |   |--- P2 W:L <= 0.06
|   |   |   |--- class: 1
|   |   |--- P2 W:L >  0.06
|   |   |   |--- P4 W:L <= 0.24
|   |   |   |   |--- P1 W:L <= 0.09
|   |   |   |   |   |--- class: 1
|   |   |   |   |--- P1 W:L >  0.09
|   |   |   |   |   |--- P3 Avg. Speak <= 16.38
|   |   |   |   |   |   |--- P2 Avg. Speak <= 26.43
|   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |--- P2 Avg. Speak >  26.43
|   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |--- P3 Avg. Speak >  16.38
|   |   |   |   |   |   |--- P3 W:L <= 0.42
|   |   |   |   |   |   |   |--- P2 Avg. Rank <= 3.08
|   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |   |--- P2 Avg. Rank >  3.08
|   |   |   |   |   |   |   |   |--- P4 W:L <= 0.19
|   |   |   |   |   |   |   |   |   |--- P4 Avg. Speak <= 25.94
|   |   |   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   | 

In [30]:
##SVM
from sklearn import svm

clf = svm.SVC(kernel = "linear")
clf.fit(X_train, y_train)

##Predict and evaluate
y_pred = clf.predict(X_test)


print("Accuracy of svm:", metrics.accuracy_score(y_test, y_pred))

Accuracy of svm: 0.5260869565217391


In [31]:
##CONCEPT CODE SAMPLES THAT WERE DEEMED NOT USEFUL FOR NOW

In [10]:
##NOT ACTUALLY USED HELPED IN DEBUGGING AND VISUALIZING DATA
from io import StringIO
import csv
# import itertools

##now need to load the dictionary as needed into csv format
# Create a string buffer to hold the CSV data
output_buffer = StringIO()

# Create a CSV writer object
csv_writer = csv.writer(output_buffer)

# Write the header row
csv_writer.writerow(['Name', 'Average Speaks', 'Average Ranks', 'Overall Win:Loss', 'Gov Win:Loss', 'Opp Win:Loss'])

# Loop through the dictionary and write each row
for name, values in db.items():
    csv_writer.writerow([name, values[3], values[4], values[5], values[6], values[7]])

# Get the CSV data as a string
csv_data = output_buffer.getvalue()
##print(csv_data)

##In list format
data = output_buffer.getvalue().splitlines()
##print(data)

# Define the output CSV file
output_file = 'output.csv'

# Open the file for writing
with open(output_file, 'w', newline='') as file:
    writer = csv.writer(file)

    # Write the header row
    writer.writerow(['Gov1 Avg. Speak', 'Gov2 Avg. Speak', 'Gov1 Avg. Rank', 'Gov2 Avg. Rank', 'Gov1 W:L', 'Gov2 W:L', 'Opp1 Avg. Speak', 'Opp2 Avg. Speak', 'Opp1 Avg. Rank', 'Opp2 Avg. Rank'])
    
    

# for combination in itertools.combinations(data[1:], 4):
#     print(combination)
#     if p == 0:
#         break
   


PermissionError: [Errno 13] Permission denied: 'output.csv'

In [ ]:
##Actuallly used, need to get actual combinations from data and fill in as need
with open('file.csv', 'r', encoding='utf-8') as csvfile:
    # Open the file for writing
    with open(output_file, 'w', newline='') as file:
        writer = csv.writer(file)

        # Write the header row of output file
        writer.writerow(['Gov1 Avg. Speak', 'Gov2 Avg. Speak', 'Gov1 Avg. Rank', 'Gov2 Avg. Rank', 'Gov1 W:L', 'Gov2 W:L', 'Opp1 Avg. Speak', 'Opp2 Avg. Speak', 'Opp1 Avg. Rank', 'Opp2 Avg. Rank'])

        ##Read as needed
        csvreader = csv.reader(csvfile)
        row = next(csvreader)

        while True:
            try:
                if row[0] == 'R':
                name1 = row[5]
                ##remove newline characters
                name1 = name1.replace('\n', ' ')
                
                name2 = row[6]
                name2 = name2.replace('\n', ' ')                  
                
                ##iterate actual table information
                row = next(csvreader)
                while row[0] != 'R':
                    ##Get 
                    
                    row = next(csvreader)
                   
                

            except StopIteration:
                break
